<a href="https://colab.research.google.com/github/VitorPaes/L3_Test/blob/main/Total_liquido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Valor líquido, dada uma base inicial

In [4]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 198 kB 56.8 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=ea90c4fb55748cb0fa9dc50cc4b36dc41dfb92c7b249ff10354c072e9907737f
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [5]:
# import base do pyspark

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, format_number, round as spark_round

sc = SparkContext()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [6]:
#constante simulando dados de entrada
TRANSACTIONS = [
    {'transacao_id': 1, 'total_bruto': 3000,  'desconto_percentual': 6.99},
    {'transacao_id': 2, 'total_bruto': 57989, 'desconto_percentual': 1.45},
    {'transacao_id': 4, 'total_bruto': 1,     'desconto_percentual': None},
    {'transacao_id': 5, 'total_bruto': 34,    'desconto_percentual': 0.0 }
]

### Transformando a lista de dados em DataFrame e organizando as colunas.



In [7]:
input_transactions = sqlContext.createDataFrame(TRANSACTIONS)
ordered_transactions = input_transactions.select('transacao_id', 'total_bruto', 'desconto_percentual')

In [8]:
ordered_transactions.show()

+------------+-----------+-------------------+
|transacao_id|total_bruto|desconto_percentual|
+------------+-----------+-------------------+
|           1|       3000|               6.99|
|           2|      57989|               1.45|
|           4|          1|               null|
|           5|         34|                0.0|
+------------+-----------+-------------------+



### Tratando valores nulos



In [9]:
transactions_notnull = ordered_transactions.fillna(0.0, subset=["desconto_percentual", "total_bruto"])

In [10]:
transactions_notnull.show()

+------------+-----------+-------------------+
|transacao_id|total_bruto|desconto_percentual|
+------------+-----------+-------------------+
|           1|       3000|               6.99|
|           2|      57989|               1.45|
|           4|          1|                0.0|
|           5|         34|                0.0|
+------------+-----------+-------------------+



### Cálculo de valor líquido


In [11]:
transactions_liquido = transactions_notnull.withColumn(
    "total_liquido", 
    spark_round((col("total_bruto") - (col("total_bruto") * (col("desconto_percentual") / 100))), 2)
)

In [12]:
transactions_liquido.show()

+------------+-----------+-------------------+-------------+
|transacao_id|total_bruto|desconto_percentual|total_liquido|
+------------+-----------+-------------------+-------------+
|           1|       3000|               6.99|       2790.3|
|           2|      57989|               1.45|     57148.16|
|           4|          1|                0.0|          1.0|
|           5|         34|                0.0|         34.0|
+------------+-----------+-------------------+-------------+



### Agregação 



In [13]:
total_liquido = transactions_liquido.agg({'total_liquido':'sum'}).select(format_number("sum(total_liquido)", 2).alias("total_liquido"))
total_liquido.show()

+-------------+
|total_liquido|
+-------------+
|    59,973.46|
+-------------+

